<a href="https://colab.research.google.com/github/ajaysuseel/MiniProject_AD/blob/main/hazard_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import json
import torch
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
from transformers import BlipProcessor, BlipForConditionalGeneration
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu

#Predefined hazard keywords for simple hazard detection

In [ ]:
HAZARD_KEYWORDS = {
    "flood": ["flood", "water", "inundated"],
    "fallen_tree": ["fallen tree", "tree"],
    "debris": ["debris", "rubble"],
    "damage": ["damaged", "cracked", "broken"]
}

#Loading captions and model

In [ ]:
def load_ground_truth(local_json_path):
    try:
        with open(local_json_path, "r") as f:
            data = json.load(f)
        # Convert the list into a dictionary: {filename: description}
        gt_data = {item["filename"]: item["description"] for item in data}
        print(f"Loaded {len(gt_data)} ground truth captions from {local_json_path}.")
        return gt_data
    except Exception as e:
        print(f"Error loading ground truth captions: {e}")
        return {}
def load_image(image_path):
    try:
        image = Image.open(image_path).convert("RGB")
        return image
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return None
def load_model_and_processor(model_path):
    try:
        processor = BlipProcessor.from_pretrained(model_path, ignore_mismatched_sizes=True)
        model = BlipForConditionalGeneration.from_pretrained(model_path, ignore_mismatched_sizes=True)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)
        model.eval()
        print(f"Model loaded on {device}.")
        return model, processor, device
    except Exception as e:
        print(f"Error loading model: {e}")
        return None, None, None


#Generate Caption for an Image

In [ ]:
def generate_caption(model, processor, device, image):
    try:
        inputs = processor(images=image, return_tensors="pt")
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            generated_ids = model.generate(**inputs)
        caption = processor.decode(generated_ids[0], skip_special_tokens=True)
        return caption
    except Exception as e:
        print(f"Error generating caption: {e}")
        return ""

#Detect Hazard in a Caption

In [ ]:
def detect_hazard(caption, hazard_keywords):
    caption_lower = caption.lower()
    detected = []
    for hazard, keywords in hazard_keywords.items():
        for kw in keywords:
            if kw in caption_lower:
                detected.append(hazard)
                break  # Found one keyword for this hazard; move to the next hazard.
    if detected:
        return ", ".join(detected)
    else:
        return "No Hazard Detected"

#Display Image with Captions, BLEU Score, and Hazard Detection Result

In [ ]:
def display_image_with_results(image_path, gt_caption, generated_caption, bleu_score, hazard_result):
    image = load_image(image_path)
    if image is None:
        print(f"Cannot display image: {image_path}")
        return
    plt.figure(figsize=(8, 6))
    plt.imshow(image)
    plt.axis('off')
    title_text = f"GT: {gt_caption}\nGen: {generated_caption}\nBLEU: {bleu_score:.4f}\nHazard: {hazard_result}"
    plt.title(title_text, fontsize=10)
    plt.show()

#Evaluate Hazard Detection Pipeline

In [ ]:
def evaluate_hazard_detection(image_folder, gt_json_path, model_path, hazard_keywords):
    # Load ground truth captions
    gt_captions = load_ground_truth(gt_json_path)
    if not gt_captions:
        print("No ground truth data available. Exiting evaluation.")
        return

    # Load the fine-tuned model and processor
    model, processor, device = load_model_and_processor(model_path)
    if model is None:
        print("Model loading failed. Exiting evaluation.")
        return

    generated_captions = {}
    image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    print(f"Found {len(image_files)} images in {image_folder}.")

    # Generate captions for each image
    for filename in tqdm(image_files, desc="Evaluating images"):
        image_path = os.path.join(image_folder, filename)
        image = load_image(image_path)
        if image is None:
            continue
        caption = generate_caption(model, processor, device, image)
        generated_captions[filename] = caption

    individual_scores = {}
    references = []
    hypotheses = {}

    # Evaluate using BLEU and run hazard detection
    for filename, gt_caption in gt_captions.items():
        if filename in generated_captions:
            gen_caption = generated_captions[filename]
            hypothesis = gen_caption.split()
            reference = [gt_caption.split()]  # BLEU expects a list of reference tokens
            score = sentence_bleu(reference, hypothesis)
            individual_scores[filename] = score
            references.append(reference)
            hypotheses[filename] = hypothesis

            # Detect hazards in the generated caption
            hazard_result = detect_hazard(gen_caption, hazard_keywords)

            # Display the image with captions, BLEU score, and hazard detection result
            image_path = os.path.join(image_folder, filename)
            display_image_with_results(image_path, gt_caption, gen_caption, score, hazard_result)
        else:
            print(f"Warning: No generated caption for {filename}")

    avg_bleu = corpus_bleu(references, list(hypotheses.values()))
    print("\n--- Evaluation Summary ---")
    for filename, score in individual_scores.items():
        print(f"{filename}: BLEU Score = {score:.4f}")
    print(f"\nAverage Corpus BLEU Score: {avg_bleu:.4f}")

#Main Execution

##Configurable Variables

In [ ]:
LOCAL_REPO_PATH = "/content/MiniProject_AD/abhiram"  # Local path to your cloned repo
IMAGE_FOLDER = os.path.join(LOCAL_REPO_PATH, "images")
GROUND_TRUTH_JSON = os.path.join(LOCAL_REPO_PATH, "captions.json")  # Ground truth captions file
MODEL_SAVE_PATH = "./models/finetuned_blip1"  # Path where your fine-tuned model is saved

In [ ]:
if __name__ == "__main__":
    evaluate_hazard_detection(IMAGE_FOLDER, GROUND_TRUTH_JSON, MODEL_SAVE_PATH, HAZARD_KEYWORDS)